# Tekstanalyse 2: Named entities regognition (NER) og Parts of Speech Tagging (POS) med SpaCy
***
***
Keywords: `context manager`, `named entity recognition`, `NER`, `parts of speech tagging`, `POS`, `WordCloud`

Nye Python-udtryk:  `with`, `.set()`, `.sorted()`, `.join()`, `WordCloud()`
***
***
I det følgende skal vi se nærmere på NLP-pakken `SpaCy`, der er indeholder effektive og kraftfulde redskaber til tekstanalyse. I modsætning til `NLTK`-pakken, der er regelbaseret, er SpaCy baseret på maskinlæring, hvilket bl.a. betyder, at den virker godt på dansk, for sprogmodulet også er trænet på danske tekster. 

SpaCy kan mange forskellige ting. Vi lægger ud med at kigge på **Named Entity Recognition** (NER) og **Parts of Speech Tagging**.

Nogle af elementerne vil være repetition af elementer fra tidligere notebooks.

Hvis der er kode sekvenser eller udtryk i ikke forstår, er det altid en god idé at bruge Google. Det kan give svar på det meste.

# 1. Forberedelse

### Dependencies
Som altid begynder vi med at importere de nødvendige `libraries`. Udover `os`, `Numpy` og `Pandas` skal vi også bruge `matplotlib` og `nltk`.

In [ ]:
import os                       # os tillader os bl.a. at finde filplaceringer på computeren
import numpy as np              # Numpy leverer noget af matematikken, der ligger under Pandas 
import pandas as pd             # Pandas tillader os at importere, oprette og manipulere data frames
import matplotlib.pyplot as plt # Importerer underbiblioteket pyplot fra pakken matplotlib
from nltk.text import Text      # nltk indholder mange forskellige funktioner, der kan bruges til tekstanalyse

Herudover skal vi også importere `SpaCy`. Når vi bruger spacy, skal vi udover at importere modulet, også loade en sprogmodel. Der er tre modeller at vælge mellem: `da_core_news_sm`, `da_core_news_md`, `da_core_news_lg`, en lille (sm), en mellem (md) og en stor (lg). Størrelsen angiver, hvor stort et korpus modellen er blevet trænet på. 

**Importér** SpaCy og **load** den store danske sprog-model. Den kører derfor lidt langsommere end de andre, men er til gengæld mere præcis. Skal man arbejde med meget store tekstmængder, kan det være en idé at bruge en mindre model. Det er en afvejning af regnekraft/tid mod præcision.

In [ ]:
import spacy          
nlp = spacy.load("da_core_news_lg")

### Import af tekster
I denne notebook skal vi arbejde med seks forskellige nytårstaler. To af Mette Frederiksen, to af Lars Løkke Rasmussen og to af Helle Thorning Schmidt. **Placér** de downloadede `.txt`-filer i en mappe i den mappe, hvori i har gemt jeres script.

#### Context manager
Der er forskellige måder at åbne filer på. Nedenfor finder I et eksempel på en såkaldt `context manager`. En af fordelene ved at bruge en context-manager er, at den automatisk lukker filerne igen efter de er blevet indlæst. 

Kodesekvensen begynder med et `for loop`, fordi vi skal åbne seks forskellige filer. Herefter følger context-manageren, der læses 'brug kommandoen 'open' til at åbne den angivne fil og gem den under variabelnavnet 'f' (f for fil er standard-notation, men det er en variabel, så navnet er valgfrit)'.

Herefter tilføjer vi talerne til listen 'Taler'. **Bemærk** at rensning foregår i samme ombæring med `.replace()`. Teksten i .txt-filerne er allerede nogenlunde rensede, og det er derfor ikke nødvendigt at bruge rense-funktionerne. Denne måde virker bedst, hvis det kun er få ting, der skal gøres. Er rensningen mere omfattende, er det mere overskueligt, at definere en pipeline-funktion.

In [ ]:
taler = [] # opretter tom liste

for fil in os.scandir(r'.\Taler'): # for-loop
    with open (fil, encoding = "utf8") as f: # context manager
        taler.append(f.read().replace("\n"," ").replace("*"," ")) # tilføj renset tekst til liste

For overskuelighedens skyld, lægger vi ud med at afprøve SpaCy på en enkelt tale. Vi gemmer talen under variabel-navnet 'tale_1'.

**Bemærk** notationen `taler[0]`. **Hvad** betyder det, og **hvordan** læses det?

In [ ]:
tale_1 = taler[0]

# SpaCy

Vi har allerede importeret `SpaCy` og load'et den store danske sprogmodel. Modellen blev gemt under navnet `nlp`. Det er standardnotation, men betegnelsen er valgfri, og I kan ændre det, hvis det er nødvendigt, fx hvis I har load'et flere forskellige modeller.

Bag navnet 'nlp' gemmer der sig nu en masse funktionalitet, på samme måde som vi tidligere har pakket funktionalitet i vores pipeline-funktioner. Når I anvender 'nlp' på en tekst, gør den derfor mange ting på en gang, og outputtet, som vi gemmer under navnet 'doc1' er et komplekst objekt, der udover teksten indholder en masse tags og meta-data.

In [ ]:
doc1 = nlp(tale_1)

Vi har ikke umiddelbart adgang til metainformationen. Prøv at taste `doc1` og `print(doc1)` i feltet nedenfor, og **diskutér** hvad I ser.

# 1.Named Entity Regonition

Det første vi skal se på er `named entity recognition`, som er en funktion, der gør det muligt at genkende personer, organisationer, steder mm.

For at hente informationen ud af vore `doc`-objekt, skal vi bruge `.ents`-kommandoen. Der er nemmest at håntere, hvis vi gemmer informationen på en liste.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [ ]:
ents1 = list(doc1.ents)

Hvis vi bare printer indholdet af variablen 'ents1', får vi blot en ordliste. Vi er derfor nødt til specifikt at bede om de enkelte elementer vha. `.text` og `.label_`-kopmmandoerne.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [ ]:
for w in ents1:
    print(w.text,w.label_)

## Personer
På listen ovenfor har vi en komplet oversigt over de ord, som SpaCy har vurderet betegner personer, steder osv. Når I skal bruge det analytiske, skal i naturligvis tjekke, om der er fejl. SpaCy performer godt, men der er pt. ingen modeller, der kan levee 100% præcision.

Hvis vi vil udtrække en liste over personer, kan vi bruge `.label_` til at sortere med.

I eksemplet nedenfor bruger vi `list comprehension`til at lave en liste, samt `.set()` til at slette dubletter, og `.sorted()` til at sortere outputtet. Et `set` (en mængde) er karakteriseret ved, at den kun indholde et eksemplar af hvert element. Når vi trnasformere en liste til en mængde, sletter vi derfor automatisk alle ord-dubletter. Kommandoen `.sorted()`transformerer herefter mængden til en sorteret liste.

Som vi så sidst fungerer `list comprehension` lidt som et `for loop`. Vi laver altså en liste af de tekstelementer (.text)fra listen 'ents1', der har et label (.label_), der er lig med "PER". Herefter slettes dubletter, og listen sorteres alfabetisk.

**Afprøv** kode sekvensen nedenfor. **Læs** koden og **diskutér** hvad de enkelte betyder.

In [ ]:
personer = sorted(set([t.text for t in ents1 if t.label_ == "PER"]))

**Print** listen for at se resultatet.

# Opgave 1
Lav tilsvarende lister, hvor i sorterer efter `"LOC"` (steder) og `"MISC"` (miscellaneous/diverse).

## Steder

## Diverse (miscellaneous)

# Opgave 2
Vælg en tale af henholdsvis Lars Løkke Rasmussen og Mette Frederiksen og lav for hver af de to taler en NER-analyse samt lister med personer, steder og diverse.

#### Lars Løkke rasmussen

#### Mette Frederiksen

# Opgave 3
**Sammenlign** listerne for de tre taler. **Diskutér** forskelle og ligheder i indholdet af talerne. **Hvordan** kan vi bruge denne information til at karakterisere teksterne.


# 3. Parts of Speech Tagging (POS)

Det næste vi skal se på er SpaCy's POS-modul. `Parts of Speech Tagging` er en funktion, der gør det muligt at opmærke alle ord med orklasse-tags. 

For at hente informationen ud af vore doc-objekt, skal vi bruge `.sents`-kommandoen. Der er ligesom sidst nemmest at håndtere, hvis vi gemmer informationen på en liste.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér** hvad de enkelte dele betyder.


In [ ]:
sents1 = list(doc1.sents)[1:]

**Hvad** er konsekvensen af at tilføje `[1:]`. **Lav** eventuelt en liste, hvor I udlader dette. Sammenlign de to lister. **Hvad** er forskellen?

Som vi gjorde ovenfor, kan vi hente tekst og tag ud af listen vha. `.text`og `.pos_`.

Kodesekvensen nedenfor giver os en koplet liste (og den er lang).

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [ ]:
for s in sents1:
    for w in s:
        print(w.text, w.pos_)

## Substantiver
Vi kan lave en en liste over substantiver ved at sortere efter POS-tagget "NOUN".

Kodesekvensen nedenfor indeholde ene kendte elementer. 


In [ ]:
subst_1 = []
for s in sents1:
    for w in s:
        if w.pos_ == "NOUN":
                subst_1.append(w.text)

print(len(subst_1))

print(sorted(set(subst_1)))

# Opgave 4
**Omskriv** kodesekvensen ovenfor, så I i stedet for et `for loop` laver en sorteret liste over substantiver ved hjælp af `list comprehension`. I kan bruge koden for NER-listerne som inspiration.

# Opgave 5

**Lav** tilsvarende lister for **verber** og **adjektiver**.

# 4. WordCloud
Der en mange måder at visualisere tekstdata på. En måderne er at lave en en `word cloud`.

For at lave en word-cloud skal vi først importere WordCloud-modulet.

Hvis I ikke har modulet installeret kan i køre følgende linje i terminalen (ikke her i Jupyter):

`pip install wordcloud`

**Importér** WordCloud-modulet.


In [ ]:
from wordcloud import WordCloud

WordCloud tager hele strenge som input. Vi skal derfor først have samlet vores liste af substantiver til en samlet `string`. Dette gøres med `.join()`-kommandoen.

In [ ]:
sub_string_1 = " ".join(subst_1)

Herefter kan vi lave en word cloud over listen af substantiver. Koden er relativt overskuelig og let at læse.

**Afprøv** sekvensen nedenfor. **Læs** koden og **diskutér**, hvad de enkelte dele betyder.

In [ ]:
cloud_sub_1 = WordCloud(width = 1200, height = 800,
                background_color ='white',
                max_words=20,
                min_font_size = 10).generate(sub_string_1)

In [ ]:
plt.imshow(cloud_sub_1, interpolation='bilinear')

# Opgave 6
Hvis I har tid tilovers skal kan I lave substantiver lister og word-clouds for nogle af de andre taler.